In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
import statistics
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

In [7]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

## Data set Air Quality

In [14]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/AirQualityUCI.csv', sep=';', decimal=',')
df = df.drop(labels=['Date','Time','Unnamed: 15', 'Unnamed: 16'], axis=1)
df.dropna(inplace=True)
#data = clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(df, 1, 1)
data.head()

,CO(GT)(t-1),PT08.S1(CO)(t-1),NMHC(GT)(t-1),C6H6(GT)(t-1),PT08.S2(NMHC)(t-1),NOx(GT)(t-1),PT08.S3(NOx)(t-1),NO2(GT)(t-1),PT08.S4(NO2)(t-1),PT08.S5(O3)(t-1),...,C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
1,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,...,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,...,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,...,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,...,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,...,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848


## Persistence method - Naive forecasting

In [9]:
# # Create a lag feature
# var = pd.DataFrame(y_appliance)
# dataframe = pd.concat([var.shift(1), var], axis=1)
# dataframe.columns = ['t', 't+1']
# dataframe

In [10]:
# Create a baseline model (Naive model)
def model_persistence(x):
  return x

def test_persistence(test_X):
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
    return predictions

## Sliding Window PyFTS:Persistence

In [11]:
def sliding_window(data,n_windows,train_size):

    result = {
        "window": [],
        "rmse": [],
        "mape": [],
        "mae": [],
        "r2": [],
        "variable":[]
    }
    
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

            print('-' * 20)
            print(f'training window {(ct)}')

            Xtrain = ttrain.loc[:,' CO conc (ppm)(t-1)':'TGS2620.3(t-1)']
            ytrain = ttrain.loc[:,' CO conc (ppm)(t)':'TGS2620.3(t)']
            Xtest = ttest.loc[:,' CO conc (ppm)(t-1)':'TGS2620.3(t-1)']
            ytest = ttest.loc[:,' CO conc (ppm)(t)':'TGS2620.3(t)']

            test_X = pd.DataFrame(Xtest.values,columns = list(df.columns))
            test_y = pd.DataFrame(ytest.values,columns = list(df.columns))
            
            for col in test_y.columns:
                forecast = test_persistence(test_X[col])
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(test_y[col], label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()

                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
                rmse = Measures.rmse(test_y[col],forecast)
                mape = Measures.mape(test_y[col],forecast)
                mae = mean_absolute_error(test_y[col], forecast)
                r2 = r2_score(test_y[col], forecast)

                result["rmse"].append(rmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)

    measures = pd.DataFrame(result)
    return measures

In [12]:
naive_result =  sliding_window(data=data,n_windows=30,train_size=0.75)

--------------------
training window 0


ValueError: Shape of passed values is (584, 18), indices imply (584, 19)

In [14]:
columns = list(df.columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = naive_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3) 



In [15]:
final_measures

,variable,rmse,mae,mape,r2
0,CO(GT),37.043,8.062,728.755,-0.336
1,PT08.S1(CO),125.831,73.727,8.184,0.588
2,NMHC(GT),18.429,9.095,3.057,0.969
3,C6H6(GT),10.767,3.595,37.687,0.487
4,PT08.S2(NMHC),152.180,97.830,11.783,0.619
5,NOx(GT),107.295,64.141,47.375,0.445
6,PT08.S3(NOx),119.237,79.816,10.990,0.686
7,NO2(GT),70.818,30.862,34.716,0.112
8,PT08.S4(NO2),170.213,98.508,8.743,0.543
9,PT08.S5(O3),184.589,127.159,14.806,0.662
